In [5]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available. Device count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
        print(f" - Memory allocated: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
        print(f" - Memory cached: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")
else:
    print("CUDA is not available. Using CPU.")

NameError: name '_C' is not defined

In [ ]:
import os

import matplotlib.pyplot as plt
import torch
import tqdm
import yaml
import gzip
import torch.nn as nn
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader
import torch.optim as optim


from torchvision import transforms
from torchvision.io import read_image

from PIL import Image


In [ ]:
class MNISTDataset(torch.utils.data.Dataset):
    def __init__(self, root, data_type='train', transform=None, target_transform=None) -> None:
        """
        Custom dataset class for MNIST data.

        :param root: Root directory where MNIST files are stored.
        :param data_type: 'train' or 't10k' to load the corresponding dataset.
        :param transform: Transformation to apply to the images.
        :param target_transform: Transformation to apply to the labels.
        """
        self.root = root
        self.data_type = data_type
        self.transform = transform
        self.target_transform = target_transform
        self.data = None
        self.targets = None
        self._load_data()

    def _load_data(self):
        """Load the MNIST dataset from gzip files using PyTorch utilities."""
        # Construct the paths for labels and images
        labels_path = os.path.join(self.root, f'{self.data_type}-labels-idx1-ubyte.gz')
        images_path = os.path.join(self.root, f'{self.data_type}-images-idx3-ubyte.gz')

        # Load the labels
        with gzip.open(labels_path, 'rb') as lbpath:
            labels = torch.tensor(list(lbpath.read()[8:]), dtype=torch.long)
            self.targets = labels

        # Load the images and reshape them to (num_samples, 28, 28)
        with gzip.open(images_path, 'rb') as imgpath:
            images = torch.tensor(list(imgpath.read()[16:]), dtype=torch.uint8)
            self.data = images.view(-1, 28, 28)  # Reshape to (num_samples, 28, 28)

    def __getitem__(self, index: int) -> tuple:
        """
        Get the item at the given index.
        :param index: The index of the item.
        :return: The transformed image and label at the index.
        """
        img, target = self.data[index], self.targets[index]

        # Apply transformations if provided
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self) -> int:
        """Return the total number of items in the dataset."""
        return len(self.data)

In [ ]:
!pwd

/mnt/c/Users/shaha/Desktop/CSE424


In [ ]:
#preprocess

transform = transforms.Compose([
    transforms.Lambda(lambda x: x.unsqueeze(0).float()),  # Add the channel dimension (1, 28, 28) and convert to float32
    transforms.Normalize((0.1307,), (0.3081,)),
    transforms.Lambda(lambda x: x.reshape(-1)),  # Flatten the image into a 1D tensor
])


# you can transform the target too if you want (e.g. one hot encode)
target_transform = transforms.Lambda(
    lambda y: y.clone().detach().long()  # Detach and convert to long type
)


In [ ]:
data_path = "/mnt/c/Users/shaha/Desktop/CSE424"

dataset = MNISTDataset(data_path, transform=transform,
                  target_transform=target_transform)

# dataset = ImageFolder(data_path, transform=transform,
#                      target_transform=target_transform)

In [ ]:
print(f'Size = {len(dataset)}')
print('Min Pixel Value: {} \nMax Pixel Value: {}'.format(dataset.data.min(), dataset.data.max()))
print('Mean Pixel Value {} \nPixel Values Std: {}'.format(dataset.data.float().mean(), dataset.data.float().std()))
print('Scaled Mean Pixel Value {} \nScaled Pixel Values Std: {}'.format(dataset.data.float().mean() / 255, dataset.data.float().std() / 255))

Size = 60000
Min Pixel Value: 0 
Max Pixel Value: 255
Mean Pixel Value 72.94035339355469 
Pixel Values Std: 90.02118682861328
Scaled Mean Pixel Value 0.28604060411453247 
Scaled Pixel Values Std: 0.35302427411079407


In [ ]:
#spliting 
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size  # Ensure the total matches the dataset size

train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size])

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")

Train size: 48000
Validation size: 12000


In [ ]:
class LinearClassifier(torch.nn.Module):
    """The linear classifier model"""

    def __init__(self, input_dim: int, num_classes: int) -> None:
        """
        Initialize the MLP model

        :param input_size: The input size
        :type input_size: int
        :param num_classes: The number of classes
        :type num_classes: int
        """
        super(LinearClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, num_classes)
        self.softmax = torch.nn.Softmax(dim=1)

    # This part is optional (Weight initialization)
    #     self.init_weights()

    # def init_weights(self):
    #     for m in self.modules():
    #         if type(m) == torch.nn.Linear:
    #             torch.nn.init.xavier_uniform_(m.weight)
    #             m.bias.data.fill_(0.01)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass

        :param x: The input tensor
        :type x: torch.Tensor

        :return: The output tensor
        :rtype: torch.Tensor
        """
        z = self.fc1(x)
        out = self.softmax(z)
        return out

In [ ]:
# Set experiment parameters
input_dim = 28 * 28
num_classes = 10
batch_sizes = [2048, 1024, 512]
learning_rates = [0.001, 0.0001]
num_epochs = 100

# Track the best configuration
best_macro_f1 = 0
best_batch_size = None
best_learning_rate = None


In [ ]:
from tqdm import tqdm  # Ensure you're importing tqdm

# Iterate through each combination of batch size and learning rate
for batch_size in batch_sizes:
    for lr in learning_rates:
        print(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}")
        
        # Create DataLoaders with the current batch size
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        # Initialize model, loss function, and optimizer
        model = LinearClassifier(input_dim=input_dim, num_classes=num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=lr)

        # Track training and validation losses
        train_losses = []
        val_losses = []
        val_f1_scores = []

        # Training loop with tqdm progress bar
        for epoch in range(num_epochs):
            model.train()  # Set model to training mode
            running_train_loss = 0.0
            
            # Wrap the train_loader with tqdm to show progress
            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} (Training)", leave=False):
                outputs = model(images)
                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                running_train_loss += loss.item()

            avg_train_loss = running_train_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            # Validation phase with tqdm progress bar
            model.eval()  # Set model to evaluation mode
            running_val_loss = 0.0
            all_preds = []
            all_labels = []
            with torch.no_grad():
                # Wrap the val_loader with tqdm to show progress
                for images, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} (Validation)", leave=False):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    running_val_loss += loss.item()
                    _, preds = torch.max(outputs, 1)  # Get predicted class labels
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            avg_val_loss = running_val_loss / len(val_loader)
            val_losses.append(avg_val_loss)

            # Calculate F1 score for validation
            val_macro_f1 = f1_score(all_labels, all_preds, average='macro')
            val_f1_scores.append(val_macro_f1)

        # Get the highest F1 score from this experiment
        max_val_f1 = max(val_f1_scores)
        if max_val_f1 > best_macro_f1:
            best_macro_f1 = max_val_f1
            best_batch_size = batch_size
            best_learning_rate = lr

        print(f"Finished Training with Batch Size: {batch_size}, Learning Rate: {lr}")
        print(f"Best Validation Macro F1 Score: {max_val_f1:.4f}\n")

        # Plot training and validation loss
        plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue')
        plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='red')
        plt.title(f"Batch Size: {batch_size}, Learning Rate: {lr}")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()

# After all experiments, print the best configuration
print(f"Best Configuration - Batch Size: {best_batch_size}, Learning Rate: {best_learning_rate}")
print(f"Best Validation Macro F1 Score: {best_macro_f1:.4f}")

# Retrain with the best configuration using the entire training dataset
train_loader = DataLoader(train_dataset_full, batch_size=best_batch_size, shuffle=True)
model = LinearClassifier(input_dim=input_dim, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=best_learning_rate)

# Train for 100 epochs
for epoch in range(100):
    model.train()
    running_train_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/100 (Final Training)"):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train_loader)

# Test the model and calculate F1 score on the test set
test_images_tensor = torch.tensor(test_images).view(-1, 28 * 28)
test_labels_tensor = torch.tensor(test_labels)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size)

# Evaluate on test set
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating on Test Set"):
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate F1 score
f1 = f1_score(all_labels, all_preds, average='macro')
print("F1 Score on Test Set:", f1)

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(conf_matrix)
disp.plot(cmap="Blues")
plt.show()
